In [4]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
from tqdm import tqdm
from datetime import datetime

url = 'https://www.vivareal.com.br/aluguel/sp/sao-jose-dos-campos/apartamento_residencial/?pagina=85'

driver = webdriver.Chrome()
driver.get(url)
sleep(4)
actions = ActionChains(driver)

errors = []
results = []

try:
    driver.find_element_by_class_name("cookie-notifier__cta").click()
except:
    pass
    
for i in tqdm(range(2)):
        
    sleep(5)
    main_div = driver.find_element_by_class_name("results-main__panel")
    properties = main_div.find_elements_by_class_name("js-property-card")
    paginator = driver.find_element_by_class_name("js-results-pagination")
    next_page = paginator.find_element_by_xpath("//a[@title='Próxima página']")
        
    for i,apartment in enumerate(properties):
        url = apartment.find_element_by_class_name("js-card-title").get_attribute("href")
        apto_id = url.split("id-")[-1][:-1]
        header = apartment.find_element_by_class_name("property-card__title").text
        address = apartment.find_element_by_class_name("property-card__address").text
        area = apartment.find_element_by_class_name("js-property-card-detail-area").text
        rooms = apartment.find_element_by_class_name("js-property-detail-rooms").text
        bathrooms = apartment.find_element_by_class_name("js-property-detail-bathroom").text
        garages = apartment.find_element_by_class_name("js-property-detail-garages").text
        try:
            amenities = apartment.find_element_by_class_name("property-card__amenities").text
        except:
            amenities = None
                # errors.append(url)
        price = apartment.find_element_by_class_name("js-property-card-prices").text
        try:
            condo = apartment.find_element_by_class_name("js-condo-price").text
        except:
            condo = None
                # errors.append(url)
            # crawler = bairro
        crawled_at = datetime.now().strftime("%Y-%m-%d %H:%M")
        results.append({"id": apto_id,
                        "url": url,
                        "header": header,
                        "address": address,
                        "area": area,
                        "rooms": rooms,
                        "bathrooms": bathrooms,
                        "garages": garages,
                        "amenities": amenities,
                        "price": price,
                        "condo": condo,
                            #"crawler": crawler,
                        "crawled_at": crawled_at})
    next_page.click()
pd.DataFrame(results).to_csv("full_results.csv", index=False)
driver.close()

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:34<00:00, 17.13s/it]
